In [23]:
import numpy as np
#from py_pcha.PCHA import PCHA
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn import mixture
from sklearn.cluster import KMeans
import os
from scipy.spatial.distance import squareform, pdist

In [10]:
def my_scaler(x):
    """standardize the predictors"""
    
    new_x = pd.DataFrame(scale(x, axis=0, with_mean=True, with_std=True, copy=True))
    new_x.columns = x.columns
    
    return new_x

def my_normal(x):
    """normalize the predictors"""
    
    scaler = MinMaxScaler() 
    scaled_values = scaler.fit_transform(x)
    new_x = pd.DataFrame(scaled_values)
    new_x.columns = x.columns

    return new_x

In [12]:
#os.chdir('/Users/yuecui/Desktop/Destiny/final')
final_cluster_clean_id=pd.read_csv("final_cluster_clean_5_10.csv")
final_cluster_clean=final_cluster_clean_id.drop(["playersID","teamID"],axis=1)

In [13]:
def cluster(dt,cluster_content,scale, cluster_number, cluster_method):
    player_feature_df=dt
    performance_cluster=[]
    weapon_cluster=[]
    
    if scale=="MinMax":
        min_max_scaler = preprocessing.MinMaxScaler()
        player_feature_df = my_normal(player_feature_df)
        
    if scale=="Standardization":
        player_feature_df = my_scaler(player_feature_df)
            
    if cluster_content=="performance":
        cluster_dt=player_feature_df[['Ability', 'Grenade', 'Melee','Super', 'assists']]
        
    if cluster_content=="weapon":
        cluster_dt=player_feature_df[['Pro Auto Rifle',
                                      'Pro Fusion Rifle', 'Pro Grenade Launcher', 'Pro Hand Cannon',
                                      'Pro Linear Fusion Rifle', 'Pro Pulse Rifle', 'Pro Rocket Launcher',
                                      'Pro Scout Rifle', 'Pro Shotgun', 'Pro Sidearm', 'Pro Sniper Rifle',
                                      'Pro Submachine Gun', 'Pro Sword', 'Pro Trace Rifle']]   
        
    if cluster_method=="GMM":
        clf = mixture.GaussianMixture(n_components=cluster_number, covariance_type='full')
        clf.fit(cluster_dt)
        cluster_dt = cluster_dt.copy()
        if cluster_content=="performance":
            performance_cluster = clf.predict(cluster_dt)
        if cluster_content=="weapon":
            weapon_cluster = clf.predict(cluster_dt) 
            
    if cluster_method=="KMeans":
        model = KMeans(n_clusters=cluster_number, n_init=30)
        model.fit(cluster_dt)
        cluster_dt = cluster_dt.copy()
        if cluster_content=="performance":
            performance_cluster = model.predict(cluster_dt)
        if cluster_content=="weapon":
            weapon_cluster = model.predict(cluster_dt)        
            
    if cluster_method=="AA":
        cluster_matrix = cluster_dt.as_matrix()
        XC, S, C, SSE, varexpl = PCHA(cluster_matrix, noc=cluster_number, delta=0.1)
        XC_df=pd.DataFrame(XC)
        if cluster_content=="performance":
            performance_cluster = XC_df.idxmax(axis=1)
        if cluster_content=="weapon":
            weapon_cluster = XC_df.idxmax(axis=1)       
            
    return performance_cluster,weapon_cluster

### Get the clustering result

In [32]:
performance_cluster = cluster(dt=final_cluster_clean,cluster_content="performance",scale="MinMax", 
        cluster_number=4, cluster_method="GMM")
performance_cluster = performance_cluster[0]

3735

In [43]:
weapon_cluster = cluster(dt=final_cluster_clean,cluster_content="weapon",scale="MinMax", 
        cluster_number=7, cluster_method="GMM")
weapon_cluster = weapon_cluster[1]
len(weapon_cluster)

3735

### Grace_Team

In [25]:
newtable = pd.read_csv("final_label_5_15.csv")

In [26]:
newset = newtable[["playersID","teamID","performance_cluster","weapon_pro_cluster","Character_Class"]]

In [27]:
newset.head()

,playersID,teamID,performance_cluster,weapon_pro_cluster,Character_Class
0,2305843009260294880,2305843009260294880230584300926065511223058430...,4,6,Warlock
1,2305843009260301047,2305843009260301047230584300926407495723058430...,3,6,Warlock
2,2305843009260304802,2305843009260304802230584300926112491023058430...,4,1,Hunter
3,2305843009260306096,2305843009260306096230584300926893816923058430...,4,3,Warlock
4,2305843009260307017,2305843009260307017230584300926409646823058430...,0,1,Hunter


** Replace with the new clustering result **

In [44]:
newset["performance_cluster"] = performance_cluster
newset["weapon_pro_cluster"] = weapon_cluster
newset.head()

/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,playersID,teamID,performance_cluster,weapon_pro_cluster,Character_Class,values
0,2305843009260294880,2305843009260294880230584300926065511223058430...,2,4,Warlock,1
1,2305843009260301047,2305843009260301047230584300926407495723058430...,2,4,Warlock,1
2,2305843009260304802,2305843009260304802230584300926112491023058430...,2,4,Hunter,1
3,2305843009260306096,2305843009260306096230584300926893816923058430...,2,2,Warlock,1
4,2305843009260307017,2305843009260307017230584300926409646823058430...,2,4,Hunter,1


### Performance + Weapon + Class

In [45]:
def teamall(df):
    df["values"] = 1
    table_weapon = df.pivot_table(index="teamID", columns="weapon_pro_cluster", values = "values",aggfunc=sum)
    table_char = df.pivot_table(index="teamID", columns="Character_Class", values = "values",aggfunc=sum)
    table_per = df.pivot_table(index="teamID", columns="performance_cluster", values = "values",aggfunc=sum)
    table_weapon = table_weapon.fillna(0)
    table_char = table_char.fillna(0)
    table_per = table_per.fillna(0)
    finalresult = pd.concat([table_weapon, table_char,table_per], axis=1).reset_index()
    return finalresult

In [46]:
all_dim = teamall(newset)
all_dim.head()

/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,teamID,0,1,2,3,4,5,6,Hunter,Titan,Warlock,0,1,2,3
0,2305843009260294880230584300926065511223058430...,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,2.0,1.0
1,2305843009260301047230584300926407495723058430...,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,3.0,0.0,2.0,2.0,0.0
2,2305843009260304802230584300926112491023058430...,0.0,2.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,4.0,0.0
3,2305843009260306096230584300926893816923058430...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,1.0,3.0,0.0
4,2305843009260307017230584300926409646823058430...,0.0,0.0,0.0,0.0,3.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,4.0,0.0


In [48]:
all_dim.columns = ['teamID', 'Weapon_0', 'Weapon_1',"Weapon_2","Weapon_3","Weapon_4","Weapon_5","Weapon_6",
                  "Hunter","Titan","Warlock",
                  "Per_0","Per_1","Per_2","Per_3"]
all_dim.head()

,teamID,Weapon_0,Weapon_1,Weapon_2,Weapon_3,Weapon_4,Weapon_5,Weapon_6,Hunter,Titan,Warlock,Per_0,Per_1,Per_2,Per_3
0,2305843009260294880230584300926065511223058430...,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,2.0,1.0
1,2305843009260301047230584300926407495723058430...,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,3.0,0.0,2.0,2.0,0.0
2,2305843009260304802230584300926112491023058430...,0.0,2.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,4.0,0.0
3,2305843009260306096230584300926893816923058430...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,1.0,3.0,0.0
4,2305843009260307017230584300926409646823058430...,0.0,0.0,0.0,0.0,3.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,4.0,0.0


### Weapon + Class

In [49]:
def teamweaponandchar(df):
    df["values"] = 1
    table_weapon = df.pivot_table(index="teamID", columns="weapon_pro_cluster", values = "values",aggfunc=sum)
    table_char = df.pivot_table(index="teamID", columns="Character_Class", values = "values",aggfunc=sum)
    table_weapon = table_weapon.fillna(0)
    table_char = table_char.fillna(0)
    finalresult = pd.concat([table_weapon, table_char], axis=1).reset_index()
    return finalresult

In [51]:
weapon_and_char = teamweaponandchar(newset)
weapon_and_char.head()

/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,teamID,0,1,2,3,4,5,6,Hunter,Titan,Warlock
0,2305843009260294880230584300926065511223058430...,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0
1,2305843009260301047230584300926407495723058430...,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,3.0
2,2305843009260304802230584300926112491023058430...,0.0,2.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0
3,2305843009260306096230584300926893816923058430...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0
4,2305843009260307017230584300926409646823058430...,0.0,0.0,0.0,0.0,3.0,0.0,1.0,2.0,1.0,1.0


### Calculate the distance matrix using city block

In [52]:
dist_wc = pd.DataFrame(squareform(pdist(weapon_and_char.iloc[:, 1:],metric = 'cityblock')), 
                     columns=weapon_and_char.teamID.unique(), index=weapon_and_char.teamID.unique())\
                    .reset_index()
dist_wc.head()

,index,2305843009260294880230584300926065511223058430092657851422305843009265925242,2305843009260301047230584300926407495723058430092908951392305843009290965137,2305843009260304802230584300926112491023058430092627147842305843009269893032,2305843009260306096230584300926893816923058430092689547272305843009284689157,2305843009260307017230584300926409646823058430092643115142305843009267824705,2305843009260309699230584300926052035523058430092659082862305843009283578425,2305843009260313050230584300926156342923058430092662893912305843009294828645,2305843009260314368230584300928558757523058430092859879742305843009294145614,2305843009260314920230584300926119480823058430092642156462305843009268588711,...,2305843009279091232230584300928338672623058430092909563352305843009291488894,2305843009279584838230584300928385805923058430092951660482305843009301313577,2305843009283349910230584300928392994323058430092901878302305843009290245928,2305843009283494720230584300928388544523058430093100742702305843009320594027,2305843009285127105230584300928628906823058430093009017302305843009320370813,2305843009285378434230584300928596688723058430093062291572305843009307125061,2305843009285938200230584300930979771623058430093105039912305843009329993983,2305843009289908109230584300929226667423058430092942876532305843009301365345,2305843009294799163230584300930078323223058430093098446092305843009318975162,2305843009309609567230584300931633481523058430093171760282305843009331015583
0,2305843009260294880230584300926065511223058430...,0.0,8.0,12.0,6.0,8.0,8.0,10.0,12.0,8.0,...,6.0,10.0,6.0,8.0,8.0,16.0,10.0,12.0,10.0,8.0
1,2305843009260301047230584300926407495723058430...,8.0,0.0,8.0,8.0,10.0,6.0,8.0,6.0,10.0,...,12.0,10.0,8.0,4.0,10.0,10.0,12.0,10.0,10.0,12.0
2,2305843009260304802230584300926112491023058430...,12.0,8.0,0.0,8.0,8.0,12.0,4.0,8.0,8.0,...,8.0,6.0,10.0,8.0,8.0,6.0,8.0,6.0,4.0,10.0
3,2305843009260306096230584300926893816923058430...,6.0,8.0,8.0,0.0,6.0,10.0,6.0,10.0,8.0,...,6.0,8.0,8.0,8.0,8.0,12.0,10.0,10.0,8.0,8.0
4,2305843009260307017230584300926409646823058430...,8.0,10.0,8.0,6.0,0.0,12.0,8.0,10.0,4.0,...,4.0,8.0,4.0,8.0,2.0,12.0,10.0,8.0,8.0,8.0


In [53]:
dist_all = pd.DataFrame(squareform(pdist(all_dim.iloc[:, 1:],metric = 'cityblock')), 
                     columns=all_dim.teamID.unique(), index=all_dim.teamID.unique())\
                     .reset_index()
dist_all.head()

,index,2305843009260294880230584300926065511223058430092657851422305843009265925242,2305843009260301047230584300926407495723058430092908951392305843009290965137,2305843009260304802230584300926112491023058430092627147842305843009269893032,2305843009260306096230584300926893816923058430092689547272305843009284689157,2305843009260307017230584300926409646823058430092643115142305843009267824705,2305843009260309699230584300926052035523058430092659082862305843009283578425,2305843009260313050230584300926156342923058430092662893912305843009294828645,2305843009260314368230584300928558757523058430092859879742305843009294145614,2305843009260314920230584300926119480823058430092642156462305843009268588711,...,2305843009279091232230584300928338672623058430092909563352305843009291488894,2305843009279584838230584300928385805923058430092951660482305843009301313577,2305843009283349910230584300928392994323058430092901878302305843009290245928,2305843009283494720230584300928388544523058430093100742702305843009320594027,2305843009285127105230584300928628906823058430093009017302305843009320370813,2305843009285378434230584300928596688723058430093062291572305843009307125061,2305843009285938200230584300930979771623058430093105039912305843009329993983,2305843009289908109230584300929226667423058430092942876532305843009301365345,2305843009294799163230584300930078323223058430093098446092305843009318975162,2305843009309609567230584300931633481523058430093171760282305843009331015583
0,2305843009260294880230584300926065511223058430...,0.0,12.0,16.0,10.0,12.0,14.0,16.0,20.0,12.0,...,8.0,14.0,10.0,14.0,10.0,24.0,16.0,18.0,18.0,12.0
1,2305843009260301047230584300926407495723058430...,12.0,0.0,12.0,10.0,14.0,10.0,10.0,10.0,10.0,...,14.0,14.0,8.0,8.0,14.0,14.0,14.0,14.0,14.0,14.0
2,2305843009260304802230584300926112491023058430...,16.0,12.0,0.0,10.0,8.0,20.0,10.0,16.0,12.0,...,12.0,12.0,14.0,16.0,10.0,14.0,14.0,14.0,12.0,12.0
3,2305843009260306096230584300926893816923058430...,10.0,10.0,10.0,0.0,8.0,16.0,10.0,16.0,10.0,...,8.0,12.0,10.0,14.0,10.0,18.0,14.0,16.0,14.0,8.0
4,2305843009260307017230584300926409646823058430...,12.0,14.0,8.0,8.0,0.0,20.0,14.0,18.0,8.0,...,8.0,14.0,8.0,16.0,4.0,20.0,16.0,16.0,16.0,10.0


### Find NN

In [55]:
def findnn(distance_df,n):
    idx = np.argsort(distance_df, axis=0)[:n]
    idx = idx.drop('index', 1)
    dfList = distance_df['index'].tolist()
    IdxDict=dict(enumerate(dfList))
    idx.replace(IdxDict,inplace=True)
    return idx

In [56]:
findnn(dist_all,5)

,2305843009260294880230584300926065511223058430092657851422305843009265925242,2305843009260301047230584300926407495723058430092908951392305843009290965137,2305843009260304802230584300926112491023058430092627147842305843009269893032,2305843009260306096230584300926893816923058430092689547272305843009284689157,2305843009260307017230584300926409646823058430092643115142305843009267824705,2305843009260309699230584300926052035523058430092659082862305843009283578425,2305843009260313050230584300926156342923058430092662893912305843009294828645,2305843009260314368230584300928558757523058430092859879742305843009294145614,2305843009260314920230584300926119480823058430092642156462305843009268588711,2305843009260314920230584300926119480923058430092642156462305843009290826059,...,2305843009279091232230584300928338672623058430092909563352305843009291488894,2305843009279584838230584300928385805923058430092951660482305843009301313577,2305843009283349910230584300928392994323058430092901878302305843009290245928,2305843009283494720230584300928388544523058430093100742702305843009320594027,2305843009285127105230584300928628906823058430093009017302305843009320370813,2305843009285378434230584300928596688723058430093062291572305843009307125061,2305843009285938200230584300930979771623058430093105039912305843009329993983,2305843009289908109230584300929226667423058430092942876532305843009301365345,2305843009294799163230584300930078323223058430093098446092305843009318975162,2305843009309609567230584300931633481523058430093171760282305843009331015583
0,2305843009260294880230584300926065511223058430...,2305843009260301047230584300926407495723058430...,2305843009260304802230584300926112491023058430...,2305843009260306096230584300926893816923058430...,2305843009260307017230584300926409646823058430...,2305843009260309699230584300926052035523058430...,2305843009260313050230584300926156342923058430...,2305843009260314368230584300928558757523058430...,2305843009260314920230584300926119480823058430...,2305843009260314920230584300926119480923058430...,...,2305843009279091232230584300928338672623058430...,2305843009279584838230584300928385805923058430...,2305843009283349910230584300928392994323058430...,2305843009283494720230584300928388544523058430...,2305843009285127105230584300928628906823058430...,2305843009285378434230584300928596688723058430...,2305843009261056190230584300927141588123058430...,2305843009289908109230584300929226667423058430...,2305843009294799163230584300930078323223058430...,2305843009309609567230584300931633481523058430...
1,2305843009266261391230584300926919968123058430...,2305843009261227120230584300926125689223058430...,2305843009266392974230584300928355955723058430...,2305843009267988584230584300926933728823058430...,2305843009262548750230584300927847536123058430...,2305843009262777930230584300926392896123058430...,2305843009261784727230584300926534937923058430...,2305843009265374690230584300926889432323058430...,2305843009266646544230584300926718111223058430...,2305843009263955015230584300926493892723058430...,...,2305843009270123375230584300929103508323058430...,2305843009262024266230584300926442353823058430...,2305843009261106294230584300926166750223058430...,2305843009260769477230584300926083111323058430...,2305843009264635080230584300926616542923058430...,2305843009268995661230584300927737580923058430...,2305843009285938200230584300930979771623058430...,2305843009262764655230584300926450113423058430...,2305843009262903065230584300926405501223058430...,2305843009261508347230584300926394508523058430...
2,2305843009270211439230584300927819019823058430...,2305843009261278313230584300926189176623058430...,2305843009265792046230584300927053230623058430...,2305843009262623050230584300926730482523058430...,2305843009267003147230584300927108431223058430...,2305843009266569389230584300926983811423058430...,2305843009261164056230584300926256779323058430...,2305843009263461321230584300929968286223058430...,23058430092607440372305843009267058780230